<a href="https://colab.research.google.com/github/shraddhapandey100/Python_Practice_With_Colab/blob/paneltime/Big_M_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1)Function that generates a matrix of correct size.
2) Functions that check if the  currentsolution is optimal.
3.)Function  that determines where a pivot element is located.
4.) Function that pivots about an element.
5.)Function  to receive string input and insert float variable into matrix.
6) Function to maximize and minimize the problem.


First, we’ll generate a numpy array with enough rows for each constraint plus the objective function and enough columns for the variables, slack variables, M (max/min) and the corresponding value.

In [ ]:
import numpy as np 
def gen_matrix(var,cons):    
    tab = np.zeros((cons+1, var+cons+2))    
    return tab

Next, we’ll check to see if 1+ pivots are required due to a negative element in the furthest right column, excluding the bottom value, of course

In [ ]:
def next_round_r(table):    
    m = min(table[:-1,-1])    
    if m>= 0:        
        return False    
    else:        
        return True

Similarly, we’ll check to see if 1+ pivots are required due to a negative element in the bottom row, excluding the final value.

In [ ]:
def next_round(table):    
    lr = len(table[:,0])   
    m = min(table[lr-1,:-1])    
    if m>=0:
        return False
    else:
        return True

And now that we’ve created functions that return booleans, whether or not additional pivots are required, we need to determine where these elements are located. We’ll start with finding negative elements in the furthest right column.

In [ ]:
def find_neg_r(table):
    lc = len(table[0,:])
    m = min(table[:-1,lc-1])
    if m<=0:        
        n = np.where(table[:-1,lc-1] == m)[0][0]
    else:
        n = None
    return n

Similarly, we need to locate negative elements in the bottom row

In [ ]:
def find_neg(table):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m<=0:
        n = np.where(table[lr-1,:-1] == m)[0][0]
    else:
        n = None
    return n

Quick recap, we’ve identified the column and row indexes, respectively, for negative elements in the last column, last row. But we need to take it one step further and find the pivot element corresponding to these values.

In [ ]:
def loc_piv_r(table):
    total = []        
    r = find_neg_r(table)
    row = table[r,:-1]
    m = min(row)
    c = np.where(row == m)[0][0]
    col = table[:-1,c]
    for i, b in zip(col,table[:-1,-1]):
        if i**2>0 and b/i>0:
            total.append(b/i)
        else:                
            total.append(10000)
    index = total.index(min(total))        
    return [index,c]
    

we’ll perform a very similar task; we need to find a pivot element corresponding to a negative element in the bottom row.

In [ ]:
def loc_piv(table):
    if next_round(table):
        total = []
        n = find_neg(table)
        for i,b in zip(table[:-1,n],table[:-1,-1]):
            if b/i >0 and i**2>0:
                total.append(b/i)
            else:
                total.append(10000)
        index = total.index(min(total))
        return [index,n]

Now, let’s finally build the pivot function! What we need it to do is pivot about an element to remove the negative entry in the final column or row and return the updated table.

In [ ]:
def pivot(row,col,table):
    lr = len(table[:,0])
    lc = len(table[0,:])
    t = np.zeros((lr,lc))
    pr = table[row,:]
    if table[row,col]**2>0:
        e = 1/table[row,col]
        r = pr*e
        for i in range(len(table[:,col])):
            k = table[i,:]
            c = table[i,col]
            if list(k) == list(pr):
                continue
            else:
                t[i,:] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        print('Cannot pivot on this element.')

With that done, now we need a means for the user to input a string, which will be converted into float variables. Our function will receive inputs such as (‘1,3,L,5’); this means 1(x1) + 3(x2) ≤ 5. Alternatively, ‘G’ could be used to mean a ≥ inequality.

In [ ]:
def convert(eq):
    eq = eq.split(',')
    if 'G' in eq:
        g = eq.index('G')
        del eq[g]
        eq = [float(i)*-1 for i in eq]
        return eq
    if 'L' in eq:
        l = eq.index('L')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq

The following is necessary to solve minimization problems. In maximization problems, M is the opposite of the objective function.

In [ ]:
def convert_min(table):
    table[-1,:-2] = [-1*i for i in table[-1,:-2]]
    table[-1,-1] = -1*table[-1,-1]    
    return table

 we need to build a function that will generate only the required number of variables x1, x2,…xn.

In [ ]:
def gen_var(table):
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    v = []
    for i in range(var):
        v.append('x'+str(i+1))
    return v

we need a means to check if 1+ constraints can be added to the matrix, meaning there are at least two rows of all 0 elements. If this condition is not satisfied, our program will not allow the user to add additional constraints.

In [ ]:
def add_cons(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0
        for j in table[i,:]:                       
            total += j**2
        if total == 0: 
            empty.append(total)
    if len(empty)>1:
        return True
    else:
        return False

The function will take the tableau as an argument as well as the equation, which will be converted using the previous function and will be inserted into the tableau, appropriately.

In [ ]:
def constrain(table,eq):
    if add_cons(table) == True:
        lc = len(table[0,:])
        lr = len(table[:,0])
        var = lc - lr -1      
        j = 0
        while j < lr:            
            row_check = table[j,:]
            total = 0
            for i in row_check:
                total += float(i**2)
            if total == 0:                
                row = row_check
                break
            j +=1
        eq = convert(eq)
        i = 0
        while i<len(eq)-1:
            row[i] = eq[i]
            i +=1        
        row[-1] = eq[-1]
        row[var+j] = 1    
    else:
        print('Cannot add another constraint.')

Similarly, we need a function that can checks to see if the objective function can be added. There’s numerous ways to approach this problem but my approach has been to add the objective function last, after all constraints have been entered, as it is the final row of the tableau.

In [ ]:
def add_obj(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0        
        for j in table[i,:]:
            total += j**2
        if total == 0:
            empty.append(total)    
    if len(empty)==1:
        return True
    else:
        return False

Similar to the constrain() function we built, we also need a function to add the objective function to the tableau, given that it satisfies add_obj().

In [ ]:
def obj(table,eq):
    if add_obj(table)==True:
        eq = [float(i) for i in eq.split(',')]
        lr = len(table[:,0])
        row = table[lr-1,:]
        i = 0        
        while i<len(eq)-1:
            row[i] = eq[i]*-1
            i +=1
        row[-2] = 1
        row[-1] = eq[-1]
    else:
        print('You must finish adding constraints before the objective function can be added.')

it’s finally time to put all the building blocks together and create the maximization and minimization functions! These functions will appear very similar, both will use while loops to determine if 1+ pivot is required, locate the pivot element, pivot about it, and continue the process until all negative elements have been removed from the last column and row. Then variables will be generated for x1 through xn and assigned values according to their positions in the tableau. Additionally, max will be assigned its appropriate value. Lastly, the function will return the max and variables in dictionary form.

In [ ]:
def maxz(table):
    while next_round_r(table)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)
    while next_round(table)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)        
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]            
            val[gen_var(table)[i]] = table[loc,-1]
        else:
            val[gen_var(table)[i]] = 0
    val['max'] = table[-1,-1]
    return val

Similarly, we need a minimization function!

In [ ]:
def minz(table):
    table = convert_min(table)
    while next_round_r(table)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)    
    while next_round(table)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)       
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]             
            val[gen_var(table)[i]] = table[loc,-1]
        else:
            val[gen_var(table)[i]] = 0 
            val['min'] = table[-1,-1]*-1
    return val

two operational tests on sample maximization and minimization problems!

In [ ]:
if __name__ == "__main__":
    # m = gen_matrix(2,2)
    # constrain(m,'2,-1,G,10')
    # constrain(m,'1,1,L,20')
    # obj(m,'5,10,0')
    # print(maxz(m))     
    # m = gen_matrix(2,4)
    # constrain(m,'2,5,G,30')
    # constrain(m,'-3,5,G,5')
    # constrain(m,'8,3,L,85')
    # constrain(m,'-9,7,L,42')
    # obj(m,'2,7,0')
    # print(minz(m))
    m = gen_matrix(5,3)
    constrain(m,'3,2,1,0,0,G,1500')
    constrain(m,'0,-1,0,1,2,G,1000')
    constrain(m,'0,0,1,-1,0,G,3000')
    
    obj(m,'2,0,2,0,2,0')
    print(minz(m))



{'x1': 0, 'min': 7000.0, 'x2': 0, 'x3': 3000.0, 'x4': 0, 'x5': 500.0}
